### 1.导入一些相关的包/库


In [ ]:
import torch                        # PyTorch 的核心库，包含张量（Tensor）操作
import torch.nn as nn               # Neural Network (神经网络) 模块，包含各种层（如全连接层、卷积层）
import torch.nn.functional as F     # 函数式接口，包含激活函数、损失函数等无参数的计算
from torch.utils.data import Dataset    # 数据集基类，用于定义"怎么读单个数据"
from torch.utils.data import DataLoader # 数据加载器，用于定义"怎么把数据打包(batch)喂给模型"
from dataclasses import dataclass   # Python 标准库，用于快速创建只包含数据的类（常用作配置参数）
import math                         # Python 标准库，包含数学运算函数
# 设置 CPU 生成随机数的种子，保证实验结果可复现
torch.manual_seed(1024)

### 2.定义一些GPT参数

In [8]:

@dataclass
class GPTConfig:
    # -----------------------------------------------------------
    # 1. 记忆力 (Context Window)
    # -----------------------------------------------------------
    block_size: int = 512
    # 含义：上下文窗口大小 / 序列最大长度
    # 通俗理解：模型一眼能看多少个字。
    # 详解：模型在预测下一个字时，最多只能回头看前 512 个字。
    #      超过这个长度，前面的内容它就"忘"了。
    #      (比如你在写第 513 个字时，它已经看不见第 1 个字了)。

    # -----------------------------------------------------------
    # 2. 学习速度/并行能力 (Parallelism)
    # -----------------------------------------------------------
    batch_size: int = 12
    # 含义：批次大小
    # 通俗理解：老师一次批改几本作业。
    # 详解：模型训练不是一句话一句话学的，而是把 12 句话捆在一起，
    #      并行地喂给 GPU 计算。这个数字越大，训练越快（但也越吃显存）。

    # -----------------------------------------------------------
    # 3. 大脑深度 (Depth)
    # -----------------------------------------------------------
    n_layer: int = 12
    # 含义：Transformer Block 的层数
    # 通俗理解：模型有多少层"过滤器"或者"专家"。
    # 详解：数据进入模型后，要经过 12 道工序的处理。
    #      层数越深，模型逻辑推理能力越强，能理解更抽象的概念。
    #      (GPT-2 Small 就是 12 层)。

    # -----------------------------------------------------------
    # 4. 思考角度 (Attention Heads)
    # -----------------------------------------------------------
    n_head: int = 12
    # 含义：多头注意力的头数
    # 通俗理解：模型在读一句话时，有多少个"心眼"同时在看。
    # 详解：比如读"苹果"这个词：
    #      第1个头关注它的颜色，第2个头关注它的形状，第3个头关注它是水果...
    #      12个头就代表它能从 12 个不同的角度去理解词与词之间的关系。

    # -----------------------------------------------------------
    # 5. 词汇理解力 (Embedding Dimension)
    # -----------------------------------------------------------
    n_embd: int = 768
    # 含义：嵌入维度 / 隐藏层大小
    # 通俗理解：用来描述一个词的"特征向量"有多长。
    # 详解：在计算机眼里，"猫"这个字不是汉字，而是一串数字。
    #      这里规定用 768 个数字来描述"猫"。
    #      数字越多（维度越高），能包含的信息就越丰富，描述得越精准。
    #      (注：通常 n_embd 除以 n_head 必须是整数，这里 768/12 = 64)。
    
    hideen_dim : int = n_embd
    # 含义：前馈神经网络的隐藏层维度
    # 通俗理解：模型在每个位置上"思考"时，能"思考"出多少个可能的答案。
    # 详解：在 Transformer 的每个位置上，都有一个小型的前馈神经网络。
    #      这个网络的隐藏层维度是 768，意味着它能"思考"出 768 个不同的可能性。
    #      维度越高，模型的表达能力越强，但计算量也越大。
    # 隐藏层是什么
    # 隐藏层是神经网络中的一个概念，位于输入层和输出层之间。
    # 它接收前一层的所有输入数据，并通过一系列的数学运算（如加权求和、激活函数等）处理这些数据，然后产生下一层的输入。
    # 在Transformer模型中，每个位置的前馈神经网络的隐藏层维度通常与嵌入维度相同，即768维。这有助于保持信息的一致性和模型的表达能力。


    # ------------------------------------------
    # 6. 防作弊机制 (Regularization)
    # -----------------------------------------------------------
    dropout: float = 0.1
    # 含义：丢弃率 (10%)
    # 通俗理解：故意让模型"失忆"或者"脑神经断连"一下。
    # 详解：在训练时，随机把 10% 的神经元关掉（置为0），不让它们工作。
    #      为什么要这么做？为了防止模型"死记硬背"（过拟合）。
    #      这逼迫模型必须学会举一反三，而不是只靠某几个神经元记答案。

    # -----------------------------------------------------------
    # 7. 每个头的视野 (Head Size)
    # -----------------------------------------------------------
    head_size: int = n_embd // n_head
    # 含义：每个注意力头的维度
    # 通俗理解：每个"心眼"能看的信息量。
    # 注意力头会做什么操作，怎么样用到head_size
    # 注意力头会根据输入的词向量，计算出每个词与其他词之间的相关性（注意力分数）。
    # 然后，它会根据这些分数，动态地调整每个词的表示，使得模型能够更好地理解词与词之间的关系。
    # head_size 决定了每个注意力头在计算注意力分数时所使用的向量维度。
    # head_size 是一个词一个词过的吗？
    # head_size 不是一个词一个词过的，而是每个注意力头在处理整个输入序列时使用的向量维度。
    # 输入序列有本质是什么
    # 输入序列本质上是一个由多个词组成的列表，每个词都被表示为一个向量（词嵌入）。
    # 详解：因为总的嵌入维度是 768，而注意力头有 12 个，
    #      所以每个头分到的维度就是 768/12=64。
    # 为什么是n_embd // n_head
    # 因为每个注意力头需要均分总的嵌入维度。
    # 数学本质：

    # -----------------------------------------------------------
    # 8. 词汇表大小
    # -----------------------------------------------------------
    #gpt2官方的tokenizer
    vocab_size: int = 50257
    # 含义：词汇表大小
    # 通俗理解：模型能认识多少个不同的词/符号。
    # 详解：GPT-2 使用的 BPE 分词器一共包含 50257 个不同的 token。
    #      这些 token 包括常见的汉字、字母、标点符号，甚至一些特殊符号。    


## GPT的结构

In [ ]:
class singleHeadAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        # 生成 Key 矩阵
        self.key = nn.Linear(config.n_embd, config.head_size)
        # key是什么？
        # 在计算注意力分数时，模型会将输入的嵌入向量通过线性变换生成 Key 矩阵。
        # 这个矩阵帮助模型理解输入序列中各个位置的信息，从而决定在计算注意力时应该关注哪些部分。
        # key用来做什么？
        # Key 矩阵用于计算注意力分数，帮助模型确定在处理当前输入时，应该关注输入序列中的哪些位置。
        # 通过与 Query 矩阵进行点积运算，模型可以计算出每个位置的重要性，从而动态地调整对不同位置的关注度。
        # nn.Linear(config.n_embd, config.head_size) 的含义
        # 这表示一个线性变换层，将输入的嵌入向量从 config.n_embd 维度映射到 config.head_size 维度。
        # config.n_embd: 嵌入维度，表示每个词的特征向量长度。
        # config.head_size: 每个注意力头的维度，表示该头能够处理的信息量。
        # 通过这个线性层，模型可以学习到如何将输入的高维信息压缩或转换为适合该注意力头处理的低维表示
        # 目的：为了让模型能够有效地计算注意力分数，从而更好地理解输入序列中的信息分布。
        self.query = nn.Linear(config.n_embd, config.head_size)
        #query是什么？
        # Query 矩阵是注意力机制中的另一个关键组成部分。
        # 它通过线性变换将输入的嵌入向量转换为 Query 矩阵。
        # query用来做什么？
        # Query 矩阵用于与 Key 矩阵进行点积运算，以计算注意力分数。
        # 这些分数反映了当前输入与输入序列中各个位置之间的相关性，帮助模型决定应该关注哪些部分的信息。

        self.value = nn.Linear(config.n_embd, config.head_size)
        # nn.Linear(config.n_embd, config.head_size) 的返回的是Callable Object，就是一个函数


        # attention_mask的新写法：通过register_buffer注册
        # 因为不用计算梯度
        # mask的作用：屏蔽掉未来的信息，只关注过去的信息
        # 2. 这个 Mask 是用来做什么的？
        # 核心目的：防止“剧透” (Prevent Peeking / Causal Masking)

        # 在训练 GPT 这种生成式模型时，我们是把整句话一次性喂进去的。比如句子是 "I love AI"。 如果不加限制，Self-Attention 机制会让每个词都能看到所有其他的词。

        # 当模型在处理 "I" 的时候，它能看到后面的 "love" 和 "AI"。

        # 这就作弊了！因为在预测时，模型说完 "I" 之后，是不应该知道后面是 "love" 的。
        

        self.register_buffer(
            'attention_mask',
            #tril是下三角矩阵
            # block_size 是文本的最大长度

            # 保留主对角线及以下的数值，把上面的全部变成 0
            torch.tril(
                # 创建一个全 1 的矩阵
                torch.ones((config.block_size, config.block_size))
            )
        )

        #dropout是什么: 随机失忆
        # 为什么要有dropout: 解决过拟合
        # Dropout的数学底层原理
        # Dropout 是一种正则化技术，通过在训练过程中随机"丢弃"（置为零）一部分神经元的输出，来防止模型过拟合。
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x: torch.Tensor):
        batch_size, seq_len, hidden_dim = x.size()
        # batch_size: 批次大小
        # seq_len: 序列长度
        # hidden_dim: 隐藏层维度 (嵌入维度)
        
        # 1. 生成 Query, Key, Value
        # [Batch, Seq, Dim] -> [Batch, Seq, Head_Size]
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        # PyTorch 的矩阵乘法 (torch.matmul 或 @) 极其“死板”，它只认最后两个维度做乘法
        # 也就是把 [Batch, Seq, Head_Size] -> [Batch, Head_Size, Seq]
        weight = q @ k.transpose(-2, -1)
        # -2 和 -1 分别表示倒数第二个和倒数第一个维度
        # 就是倒数第二个和倒数第一个维度转置吗？
        # weights 的形状是 (batch_size, seq_len, seq_len)
        weight = weight.masked_fill(
            # [:seq_len, :seq_len] 是因为实际的序列长度可能小于 block_size
            # 意为着只取前 seq_len 行和前 seq_len 列，其余位置填充为 -inf
            self.attention_mask[:seq_len, :seq_len] == 0, 
            float('-inf')
            )

        #记得weight除以sqrt(d_k)
        # 为什么要除以这个 8？（为什么要缩放？）
        #这是为了救 Softmax 一命，防止梯度消失。
        weight = weight / math.sqrt(self.head_size)
        weight = F.softmax(weight, dim=-1)

        #Attention Dropout要放在weight权重化前面
        weight = self.dropout(weight)
        output = weight @ v  # [Batch, Seq, Head_Size]
        return output

